# ResonFit: Basic Usage Example

This notebook demonstrates how to use the ResonFit package to analyze superconducting resonator S21 data.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Import core ResonFit components
from resonfit.core.pipeline import ResonatorPipeline
from resonfit.preprocessing import CableDelayCorrector, AmplitudePhaseNormalizer
from resonfit.fitting import DCMFitter
from resonfit.visualization import ResonancePlotter

## 1. Loading Data

First, let's load some S21 data. This could be from a CSV file or any other source.

In [ ]:
# Example using a CSV file with columns: frequency, real, imaginary
def load_data(filename, skip_header=3):
    try:
        data = np.genfromtxt(filename, skip_header=skip_header, delimiter=',')
        freqs = data[:, 0]  # Hz
        s21 = data[:, 1] + 1j * data[:, 2]  # Complex S21
        return freqs, s21
    except Exception as e:
        print(f"Error loading data: {e}")
        return None, None

# Load your data here
# freqs, s21 = load_data("your_data.csv")

# For demonstration purposes, let's generate synthetic data
def generate_sample_data(noise_level=0.01):
    fr = 5e9  # 5 GHz resonance frequency
    Ql = 10000  # Loaded quality factor
    Qc = 20000  # Coupling quality factor
    phi = 0.1  # Impedance mismatch angle
    
    # Generate frequencies centered around fr
    freqs = np.linspace(fr - fr/Ql*5, fr + fr/Ql*5, 401)
    
    # Delay and background
    delay = 50e-9  # 50 ns
    a = 0.9
    alpha = 0.2
    
    # Generate ideal S21 data
    Qc_complex = Qc * np.exp(-1j * phi)
    s21_ideal = 1 - (Ql / Qc_complex) / (1 + 2j * Ql * (freqs - fr) / fr)
    
    # Add environmental effects
    s21_with_env = a * np.exp(1j * alpha) * s21_ideal * np.exp(1j * 2 * np.pi * freqs * delay)
    
    # Add noise
    noise = noise_level * (np.random.randn(len(freqs)) + 1j * np.random.randn(len(freqs)))
    s21_noisy = s21_with_env + noise
    
    return freqs, s21_noisy, {"fr": fr, "Ql": Ql, "Qc": Qc, "phi": phi}

# Generate sample data
freqs, s21, true_params = generate_sample_data(noise_level=0.005)

## 2. Visualizing Raw Data

In [ ]:
# Create a plotter instance
plotter = ResonancePlotter()

# Plot the raw data
plotter.plot_raw_data(freqs, s21, title="Sample Resonator Data");

## 3. Setting Up the Analysis Pipeline

Now let's set up a pipeline to process our data.

In [ ]:
# Create preprocessing components
delay_corrector = CableDelayCorrector()
normalizer = AmplitudePhaseNormalizer()

# Create fitter
dcm_fitter = DCMFitter(weight_bandwidth_scale=1.0)

# Set up pipeline
pipeline = ResonatorPipeline()
pipeline.add_preprocessor(delay_corrector)
pipeline.add_preprocessor(normalizer)
pipeline.set_fitter(dcm_fitter)

## 4. Running the Pipeline

Now let's run the pipeline to analyze our data.

In [ ]:
# Run the pipeline
results = pipeline.run(freqs, s21, plot=True)

# Print the results
print("\nFitting Results:")
print(f"Resonance Frequency: {results['fr']/1e9:.6f} GHz")
print(f"Loaded Quality Factor (Ql): {results['Ql']:.1f}")
print(f"Internal Quality Factor (Qi): {results['Qi']:.1f}")
print(f"Coupling Quality Factor (|Qc|): {results['Qc_mag']:.1f}")
print(f"Impedance Mismatch Angle (phi): {results['phi']:.4f} rad")
print(f"RMSE: {results['rmse']:.3e}")

# Compare with true values if using generated data
print("\nComparison with True Values:")
print(f"Resonance Frequency: {true_params['fr']/1e9:.6f} GHz (True) vs {results['fr']/1e9:.6f} GHz (Fitted)")
print(f"Loaded Quality Factor: {true_params['Ql']:.1f} (True) vs {results['Ql']:.1f} (Fitted)")
print(f"Phi: {true_params['phi']:.4f} rad (True) vs {results['phi']:.4f} rad (Fitted)")

## 5. Advanced Visualization

Let's use the visualization module to create more detailed plots.

In [ ]:
# 수정된 코드 (방법 1: 튜플 언패킹)
freqs_delay_corrected, s21_delay_corrected = pipeline.get_intermediate_data(0)
freqs_normalized, s21_normalized = pipeline.get_intermediate_data(1)
model_data = dcm_fitter.get_model_data()

# Plot delay correction
optimal_delay = delay_corrector.get_delay()
plotter.plot_delay_correction(
    freqs, s21, s21_delay_corrected,
    delay_ns=optimal_delay*1e9
)

In [ ]:
# Plot fitting results
plotter.plot_fitting_results(
    freqs, s21_normalized, model_data, results['fr'],
    title=f"DCM Fit: Qi={results['Qi']:.1f}, fr={results['fr']/1e9:.6f} GHz"
)

## 6. Extracting Q Factors at Different Powers

In a real experiment, you'd typically measure the resonator at different power levels to study the power dependence of the internal Q factor. Here's how you would analyze such data:

In [ ]:
# Example: Simulate data at different power levels
# This would be replaced with your actual measurements
np.random.seed(42)
power_levels = np.array([-70, -65, -60, -55, -50, -45, -40])  # dBm
# Convert to average photon numbers (arbitrary scaling for example)
photon_numbers = 10**(power_levels/10) * 1e-3  

# Simulate how Qi might change with power (TLS effects)
Qi_values = 1e4 * (1 + photon_numbers/min(photon_numbers))**0.3

# Create a list to store results
power_sweep_results = []

# Process each power level
for i, (power, photons, Qi) in enumerate(zip(power_levels, photon_numbers, Qi_values)):
    # Generate synthetic data for this power level
    _, s21_power, _ = generate_sample_data(noise_level=0.005 * np.sqrt(1/photons))
    
    # We would replace this with actual measurements at different powers
    # E.g., freqs, s21_power = load_data(f"resonator_data_{power}dBm.csv")
    
    # Run the pipeline
    results_power = pipeline.run(freqs, s21_power, plot=False)
    
    # Store results
    power_sweep_results.append({
        'power_dBm': power,
        'photon_number': photons,
        'fr': results_power['fr'],
        'Qi': results_power['Qi'],
        'Ql': results_power['Ql'],
        'Qc_mag': results_power['Qc_mag'],
        'phi': results_power['phi'],
        'rmse': results_power['rmse']
    })

# Extract data for plotting
powers = [r['power_dBm'] for r in power_sweep_results]
photons = [r['photon_number'] for r in power_sweep_results]
Qis = [r['Qi'] for r in power_sweep_results]
frs = [r['fr'] for r in power_sweep_results]

# Plot Qi vs photon number
plt.figure(figsize=(10, 6))
plt.semilogx(photons, Qis, 'o-', markersize=8)
plt.xlabel('Average Photon Number')
plt.ylabel('Internal Quality Factor (Qi)')
plt.title('Power Dependence of Internal Quality Factor')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Print table of results
print("Power Sweep Results:")
print("-" * 80)
print(f"{'Power (dBm)':12s} {'Photons':12s} {'fr (GHz)':12s} {'Qi':12s} {'Qc':12s} {'phi (rad)':12s}")
print("-" * 80)
for r in power_sweep_results:
    print(f"{r['power_dBm']:12.1f} {r['photon_number']:12.1e} {r['fr']/1e9:12.6f} {r['Qi']:12.1f} {r['Qc_mag']:12.1f} {r['phi']:12.4f}")

## 7. Conclusion

This basic example demonstrates how to use the ResonFit package to analyze resonator data. The modular design allows for easy customization and extension. For more advanced usage, please refer to the other example notebooks and the API documentation.